In [3]:
import csv
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [82]:
#setup data

#define risk free rate(rfr) as the three month treasure yield (DGS3MO)
rfr = np.array((5244,1))
#define term spread as 10 year treasure interest(DGS10) minus 1 year treasure interest(DGS1)
ts = np.array((5244,1))
#define default spread as the yield gap between AAA(DAAA) and BBB(DBAA) grade bonds
ds = np.array((5244,1))
ones = np.ones((5243,1))[:,0]

data1 = pd.read_csv('spread.csv')
data2 = pd.read_csv('StockMarket.csv')
data = pd.merge(data1, data2, on='Date')
rfr = data['DGS3MO']
ds = data['DGS10'] - data['DGS1']
ts = data['DBAA'] - data['DAAA']
# A = np.stack((rfr, ds, ts, ones), axis = 1)

rfr2=np.zeros((5243,))
ds2=np.zeros((5243,))
ts2=np.zeros((5243,))
for i in range(5243):
    rfr2[i] = rfr[i+1]-rfr[i]
    ds2[i] = ds[i+1]-ds[i]
    ts2[i] = ts[i+1]-ts[i]

#the yield of three major index: Dow Jones, SP 500 and Nasdaq
B_raw = np.stack((data['DJIA'], data['SP500'], data['NASDAQ']), axis = 1)
#Labeled as -1 if the market loss the next day, +1 otherwise 
B = np.zeros((5244, 3))
for i in range(5244):
    for j in range(3):
        B[i][j] = -1 if B_raw[i][j] < 0 else 1

A = np.stack((rfr[1:], ds[1:], ts[1:], rfr2, ds2, ts2, B_raw[:5243,0], B_raw[:5243,1], B_raw[:5243,2], ones), axis = 1)
# A = np.stack((rfr2, ds2, ts2, ones), axis = 1)
A = preprocessing.normalize(A)

#delete first label since there is no previous day data to predict
B= np.delete(B, 0, 0)
n = A.shape[1]#number of feature
nodes = 7#number of nodes in each layer
# print(n)

In [83]:
#module
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.l1 = nn.Linear(n, nodes)
#         self.l2 = nn.Linear(nodes, nodes)
#         self.l3 = nn.Linear(nodes, nodes)
#         self.l4 = nn.Linear(nodes, nodes)
        self.l5 = nn.Linear(nodes, 3)
    
    def forward(self, inputs):
        x = F.relu(self.l1(inputs))
#         x = F.relu(self.l2(x))
#         x = F.relu(self.l3(x))
#         x = F.relu(self.l4(x))
        x = self.l5(x)
        return x

In [84]:
#custom loss function
# def HingeLoss(my_outputs, my_labels):
#     #specifying the batch size
#     size = my_outputs.size()[0] 
#     hingeloss = torch.zeros([size, 3], requires_grad = True)
#     my_outputs = torch.sign(my_outputs)
# #     for k in range(size):
# #         for l in range(3):
# #             hingeloss[k][l] = max(0, 1 - my_outputs[k][l]*my_labels[k][l])
# #     for k in range(3):
# #         hingeloss[:, k] = torch.nn.Threshold(0, 0)(1- torch.dot(my_outputs[:,k], my_labels[:,k]))
#     v = torch.ones([size, 3])
#     hingeloss = my_outputs@my_labels

#     #returning the results
# #     return torch.sum(hingeloss) / size
#     return hingeloss.mean()

In [85]:
X = A
Y = B

sqer1 = np.zeros((10))
er_rate1 = np.zeros((10))
sqer2 = np.zeros((10))
er_rate2 = np.zeros((10))
sqer3 = np.zeros((10,))
er_rate3 = np.zeros((10))

In [86]:
#10-fold CV

for j in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle = True, random_state=i)
    X_train_tensor = torch.from_numpy(X_train).float().cuda()
    Y_train_tensor = torch.from_numpy(Y_train).float().cuda()
    X_test_tensor = torch.from_numpy(X_test).float().cuda()
    Y_test_tensor = torch.from_numpy(Y_test).float().cuda()
    
    epochs = 6000
    LEARNING_RATE = 0.001

    model = binaryClassification()
    model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

#     tic = time.perf_counter()
    model.train()
    for i in range(epochs):
        optimizer.zero_grad()
        Y_pred = model(X_train_tensor)
    #     Y_pred = torch.sign(Y_pred)
        loss = nn.L1Loss()(Y_pred, Y_train_tensor)
#         loss = HingeLoss(Y_pred, Y_train_tensor)
        loss.backward()
        optimizer.step()
#         print(loss)
#     toc = time.perf_counter()
#     print(toc - tic)
    print(loss)
    
    model.eval()
    with torch.no_grad():
        Y_test_pred = model(X_test_tensor)
        Y_test_pred = torch.sign(Y_test_pred)
        er = sum(abs(Y_test_pred - Y_test_tensor))
#         print(er / (Y_test.shape[0] * 2))
        sqer1[j] = torch.norm((Y_test_pred[:,0] - Y_test_tensor[:,0]))
        er_rate1[j] = er[0] / (Y_test.shape[0] * 2)
        sqer2[j] = torch.norm((Y_test_pred[:,1] - Y_test_tensor[:,1]))
        er_rate2[j] = er[1] / (Y_test.shape[0] * 2)
        sqer3[j] = torch.norm((Y_test_pred[:,2] - Y_test_tensor[:,2]))
        er_rate3[j] = er[2] / (Y_test.shape[0] * 2)



tensor(0.8700, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8695, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8778, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8975, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8995, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8824, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8716, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8731, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8992, device='cuda:0', grad_fn=<L1LossBackward>)
tensor(0.8878, device='cuda:0', grad_fn=<L1LossBackward>)


In [87]:
print(np.round(sqer1.mean() / Y_test.shape[0], 4), np.round(er_rate1.mean(), 3))
print(np.round(sqer2.mean() / Y_test.shape[0], 4), np.round(er_rate2.mean(), 3))
print(np.round(sqer3.mean() / Y_test.shape[0], 4), np.round(er_rate3.mean(), 3))

0.055 0.397
0.0553 0.402
0.0554 0.403
